<a href="https://colab.research.google.com/github/yoshimethod/bioinformatics/blob/main/bioinformatics_pr2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.7 MB/s eta 0:00:00


In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import sklearn

In [6]:
gse_mRNA_exprs_normal_selected=pd.read_csv("/content/drive/MyDrive/GSE36376_normal.csv", index_col=0)
gse_mRNA_exprs_tumor_selected=pd.read_csv("/content/drive/MyDrive/GSE36376_tumor.csv", index_col=0)

In [8]:
gse_mRNA_exprs_normal_selected.loc['class']=0
gse_mRNA_exprs_tumor_selected.loc['class']=1

In [9]:
from sklearn.model_selection import train_test_split

In [11]:
gse_mRNA_exprs_normal_selected_train, gse_mRNA_exprs_normal_selected_test=train_test_split(gse_mRNA_exprs_normal_selected.T, train_size=0.75, random_state=0)
gse_mRNA_exprs_tumor_selected_train, gse_mRNA_exprs_tumor_selected_test=train_test_split(gse_mRNA_exprs_tumor_selected.T, train_size=0.75, random_state=0)

In [13]:
gse_mRNA_exprs_train=pd.concat([gse_mRNA_exprs_normal_selected_train, gse_mRNA_exprs_tumor_selected_train])
X_train=gse_mRNA_exprs_train.iloc[:,0:-1]
y_train=gse_mRNA_exprs_train.iloc[:, -1]
gse_mRNA_exprs_test=pd.concat([gse_mRNA_exprs_normal_selected_test, gse_mRNA_exprs_tumor_selected_test])
X_test =gse_mRNA_exprs_test.iloc[:, 0:-1]
y_test =gse_mRNA_exprs_test.iloc[:, -1]

In [14]:
X_train.describe()

Symbol,MT1G,HAMP,MT1F,FCN3,CYP3A4,MT1H,CYP1A2,SPINK1,AKR1B10,GPC3,MT1M,TOP2A,CLEC1B
count,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000
mean,11.592484,10.679672,10.358558,9.992271,9.743794,9.522384,9.430548,9.323191,9.000996,8.362934,8.289994,7.536580,7.138975
std,2.726317,2.769697,2.457899,2.524754,2.701083,2.720323,3.003620,3.035246,2.719765,2.422019,2.163480,1.618519,1.624832
min,6.421117,6.579506,6.287872,5.759413,5.541597,5.650433,5.646007,5.672313,5.713809,5.505577,5.515545,5.487845,5.263245
25%,9.001321,7.802423,7.757092,7.336304,6.684946,6.781180,6.021638,6.488306,6.487949,6.130235,6.043194,6.037377,5.673154
50%,12.690336,10.483947,10.850347,10.600522,10.551383,9.946680,9.765023,8.223943,8.239076,7.370867,8.027586,6.981059,6.528087
75%,14.145431,13.606179,12.779727,12.393297,12.225083,12.136667,12.416782,12.724850,11.647682,10.798906,10.326342,9.043164,8.625387
max,14.604863,14.395189,14.350171,13.684159,14.161284,14.248271,14.019988,14.501329,14.195891,13.370585,14.324015,11.160612,11.374832
